not working dont know why

In [ ]:
train['label'] = 0
test['label']  = 1
trte = pd.concat([train,test],axis=0,ignore_index=True)

import lightgbm as lgb
import time
from sklearn.model_selection import KFold,StratifiedKFold

def validation_prediction_lgb(X,y,feature_names, ratio =1, X_test = None,istest = False):
    n_fold = 5
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    params = {
    'bagging_freq': 5, 
    'boost_from_average':'false',
    'boost': 'gbdt',
    'learning_rate': 0.01,
    'max_depth': 5,
    'metric':'auc',
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10.0,
    'tree_learner': 'serial',
    'objective': 'binary',
    'verbosity': 1}

    importances = pd.DataFrame() 

    if istest:
        prediction = np.zeros(len(X_test))
    models = []

    for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X[train_index], X[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
        weights = [ratio  if val == 1 else 1 for val in y_train]
        train_data = lgb.Dataset(X_train, label=y_train,  weight=weights)
        valid_data = lgb.Dataset(X_valid, label=y_valid)
        model = lgb.train(params,train_data,num_boost_round=20000,
                        valid_sets = [train_data, valid_data],verbose_eval=200,early_stopping_rounds = 200)

        imp_df = pd.DataFrame() 
        imp_df['feature']  = feature_names
        imp_df['split']    = model.feature_importance()
        imp_df['gain']     = model.feature_importance(importance_type='gain')
        imp_df['fold']     = fold_n + 1
        importances = pd.concat([importances, imp_df], axis=0)
        models.append(model)

        if istest == True:
            prediction += model.predict(X_test, num_iteration=model.best_iteration)/5
    if istest == True:     
        return models,importances, prediction
    else:
        return models,importances
    
train_cols = [col for col in trte.columns if col not in ['target','ID_code',\
                            'label','grade','subGrade','issueDate','earliesCreditLine']]
print(len(train_cols))

models,importances = validation_prediction_lgb(trte[train_cols].values, trte['label'].values, train_cols)